In [1]:
!pip install -q datasets transformers peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.0 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

In [5]:
dataset = load_dataset('Abirate/english_quotes')

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

In [16]:
dataset_split = dataset['train'].train_test_split(test_size=0.1)
dataset_split

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2257
    })
    test: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 251
    })
})

In [29]:
train_dataset = dataset_split['train']
val_dataset = dataset_split['test']

In [18]:
model_name = "gpt2"

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [21]:
tokenizer.pad_token = tokenizer.eos_token

In [22]:
tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [23]:
len(tokenizer)

50257

In [27]:
train_dataset['quote'][1]

"“Sometimes I can hear my bones straining under the weight of all the lives I'm not living.”"

In [30]:
def tokenize(batch):
  tokenized = tokenizer(batch['quote'], padding="max_length", trucation=True)
  tokenized['labels'] = tokenized['input_ids'].copy()
  return tokenized


train_data = train_dataset.map(tokenize, batched=True)
val_data = val_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2257 [00:00<?, ? examples/s]

Map:   0%|          | 0/251 [00:00<?, ? examples/s]

In [34]:
train_data['quote'][1]

"“Sometimes I can hear my bones straining under the weight of all the lives I'm not living.”"

In [33]:
train_data

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2257
})

In [ ]:
(train_data[10]['input_ids'])

In [37]:
tokenizer.decode(train_data[0]['input_ids'])

'“To err is human, to forgive, divine.”<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|

In [42]:
tokenizer.decode(
    train_data[10]['input_ids'],
    skip_special_tokens=True
)

'“And, when you want something, all the universe conspires in helping you to achieve it.”'

In [43]:
# LOAD MODEL

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.float16,
                                             device_map ="auto"
                                             )

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [44]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [45]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ["c_attn"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model_peft = get_peft_model(model, lora_config)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [46]:
model_peft

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
          

In [47]:
training_args = TrainingArguments(
    output_dir = "./lora_fine_tune",
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 2,
    eval_strategy = "steps",
    eval_steps = 50,
    logging_steps = 50,
    learning_rate = 2e-4,
    save_steps = 50,
    num_train_epochs = 5,
    fp16 = True,
    report_to = 'none'

)

In [49]:
trainer = Trainer(
    model = model_peft,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,5.829203,0.271721
100,0.234475,0.162204
150,0.170620,0.155311
200,0.168213,0.150151
250,0.186832,0.145585
300,0.158431,0.141889
350,0.134379,0.139218
400,0.163143,0.137566
450,0.180447,0.135289
500,0.144204,0.132971


TrainOutput(global_step=1415, training_loss=0.3542131968184832, metrics={'train_runtime': 1482.44, 'train_samples_per_second': 7.612, 'train_steps_per_second': 0.955, 'total_flos': 5917808897556480.0, 'train_loss': 0.3542131968184832, 'epoch': 5.0})

In [50]:
model_peft.save_pretrained('lora-gpt2')
tokenizer.save_pretrained('lora-gpt2')

('lora-gpt2/tokenizer_config.json', 'lora-gpt2/tokenizer.json')

In [51]:
# inference form saved model

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# load tokenizer

base_model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained("lora-gpt2")
tokenizer.pad_token = tokenizer.eos_token

# load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    dtype= torch.float16,
    device_map='auto'
)

# load lora adapter
model = PeftModel.from_pretrained(
    base_model,
    'lora-gpt2'
)

# Build pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    dtype = torch.float16,
    device_map = 'auto'
    )


Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [54]:
prompt = "The secret of happiness is "

outputs = text_generation_pipeline(prompt, max_new_tokens=100, num_return_sequences=1, do_sample=True, temperature=0.7)
print(outputs[0]['generated_text'])

Both `max_new_tokens` (=100) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The secret of happiness is  to find it in your heart. It is to find it in your soul. It is to find it in yourself. It is to find it in your heart. It is to find it in your heart. It is to find it in your heart. It is to find it in your heart. It is to find it in your heart. It is to find it in your heart. It is to find it in your heart. It is to find it in your heart. It is to


In [55]:
prompt = "Once upom a time"

outputs = text_generation_pipeline(prompt, max_new_tokens=100, num_return_sequences=1, do_sample=True, temperature=0.7)
print(outputs[0]['generated_text'])

Both `max_new_tokens` (=100) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Once upom a time to reflect on our past, we should remember that our future is a bigger picture. We must be mindful of what we have accomplished in the past. We must remember that our children are also capable of future achievements, and we must remember that our children also deserve to be grateful for being made to feel that way. We must be grateful for our parents. We must be thankful for the kindness and strength of our family. We must be grateful for the kindness and strength of our community. We must be
